In [1]:
!pip install "sagemaker>=2.190.0"  "huggingface_hub" "opencv-python" --upgrade --quiet

In [2]:
import sagemaker
import boto3
import tarfile
import os
from sagemaker.s3 import S3Uploader
from sagemaker.huggingface.model import HuggingFaceModel

from distutils.dir_util import copy_tree
from distutils.file_util import copy_file
from pathlib import Path
from huggingface_hub import snapshot_download, hf_hub_download
import random

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.0' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [3]:
sess = sagemaker.Session()
# sagemaker session bucket -> used for uploading data, models and logs
# sagemaker will automatically create this bucket if it not exists
sagemaker_session_bucket=None
if sagemaker_session_bucket is None and sess is not None:
    # set to default bucket if a bucket name is not given
    sagemaker_session_bucket = sess.default_bucket()

try:
    role = sagemaker.get_execution_role()
except ValueError:
    iam = boto3.client('iam')
    role = iam.get_role(RoleName='sagemaker_execution_role')['Role']['Arn']

sess = sagemaker.Session(default_bucket=sagemaker_session_bucket)

print(f"sagemaker role arn: {role}")
print(f"sagemaker bucket: {sess.default_bucket()}")
print(f"sagemaker session region: {sess.boto_region_name}")

sagemaker role arn: arn:aws:iam::564976835481:role/service-role/AmazonSageMakerServiceCatalogProductsUseRole
sagemaker bucket: sagemaker-eu-north-1-564976835481
sagemaker session region: eu-north-1


In [4]:
base_model_path = f"SG161222/Realistic_Vision_V4.0_noVAE"
vae_model_path = f"stabilityai/sd-vae-ft-mse"
image_encoder_path = f"laion/CLIP-ViT-H-14-laion2B-s32B-b79K"
ip_ckpt = "ip-adapter-faceid-plus_sd15.bin"

In [5]:
!rm -drf model*

In [6]:
# create model dir
model_tar = Path(f"model-{random.getrandbits(16)}")
model_tar.mkdir(exist_ok=True)

In [7]:
# download snapshot
snapshot_dir = snapshot_download(repo_id=base_model_path, allow_patterns='text_encoder/*')
# copy snapshot to model dir
copy_tree(snapshot_dir, str(model_tar.joinpath(base_model_path)))

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

['model-13382/SG161222/Realistic_Vision_V4.0_noVAE/.gitattributes',
 'model-13382/SG161222/Realistic_Vision_V4.0_noVAE/README.md',
 'model-13382/SG161222/Realistic_Vision_V4.0_noVAE/text_encoder/config.json',
 'model-13382/SG161222/Realistic_Vision_V4.0_noVAE/text_encoder/model.safetensors',
 'model-13382/SG161222/Realistic_Vision_V4.0_noVAE/unet/config.json',
 'model-13382/SG161222/Realistic_Vision_V4.0_noVAE/unet/diffusion_pytorch_model.safetensors',
 'model-13382/SG161222/Realistic_Vision_V4.0_noVAE/tokenizer/special_tokens_map.json',
 'model-13382/SG161222/Realistic_Vision_V4.0_noVAE/tokenizer/tokenizer_config.json',
 'model-13382/SG161222/Realistic_Vision_V4.0_noVAE/tokenizer/merges.txt',
 'model-13382/SG161222/Realistic_Vision_V4.0_noVAE/tokenizer/vocab.json',
 'model-13382/SG161222/Realistic_Vision_V4.0_noVAE/model_index.json']

In [8]:
# download snapshot
snapshot_dir = snapshot_download(repo_id=base_model_path, allow_patterns='unet/*')
# copy snapshot to model dir
copy_tree(snapshot_dir, str(model_tar.joinpath(base_model_path)))

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

['model-13382/SG161222/Realistic_Vision_V4.0_noVAE/.gitattributes',
 'model-13382/SG161222/Realistic_Vision_V4.0_noVAE/README.md',
 'model-13382/SG161222/Realistic_Vision_V4.0_noVAE/text_encoder/config.json',
 'model-13382/SG161222/Realistic_Vision_V4.0_noVAE/text_encoder/model.safetensors',
 'model-13382/SG161222/Realistic_Vision_V4.0_noVAE/unet/config.json',
 'model-13382/SG161222/Realistic_Vision_V4.0_noVAE/unet/diffusion_pytorch_model.safetensors',
 'model-13382/SG161222/Realistic_Vision_V4.0_noVAE/tokenizer/special_tokens_map.json',
 'model-13382/SG161222/Realistic_Vision_V4.0_noVAE/tokenizer/tokenizer_config.json',
 'model-13382/SG161222/Realistic_Vision_V4.0_noVAE/tokenizer/merges.txt',
 'model-13382/SG161222/Realistic_Vision_V4.0_noVAE/tokenizer/vocab.json',
 'model-13382/SG161222/Realistic_Vision_V4.0_noVAE/model_index.json']

In [9]:
# download snapshot
snapshot_dir = snapshot_download(repo_id=base_model_path, allow_patterns='tokenizer/*')
# copy snapshot to model dir
copy_tree(snapshot_dir, str(model_tar.joinpath(base_model_path)))

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

['model-13382/SG161222/Realistic_Vision_V4.0_noVAE/.gitattributes',
 'model-13382/SG161222/Realistic_Vision_V4.0_noVAE/README.md',
 'model-13382/SG161222/Realistic_Vision_V4.0_noVAE/text_encoder/config.json',
 'model-13382/SG161222/Realistic_Vision_V4.0_noVAE/text_encoder/model.safetensors',
 'model-13382/SG161222/Realistic_Vision_V4.0_noVAE/unet/config.json',
 'model-13382/SG161222/Realistic_Vision_V4.0_noVAE/unet/diffusion_pytorch_model.safetensors',
 'model-13382/SG161222/Realistic_Vision_V4.0_noVAE/tokenizer/special_tokens_map.json',
 'model-13382/SG161222/Realistic_Vision_V4.0_noVAE/tokenizer/tokenizer_config.json',
 'model-13382/SG161222/Realistic_Vision_V4.0_noVAE/tokenizer/merges.txt',
 'model-13382/SG161222/Realistic_Vision_V4.0_noVAE/tokenizer/vocab.json',
 'model-13382/SG161222/Realistic_Vision_V4.0_noVAE/model_index.json']

In [10]:
# download snapshot
snapshot_dir = hf_hub_download(repo_id=base_model_path, filename="model_index.json")
# copy snapshot to model dir
copy_file(snapshot_dir, str(model_tar.joinpath(base_model_path)))

('model-13382/SG161222/Realistic_Vision_V4.0_noVAE/model_index.json', 1)

In [11]:
# download snapshot
snapshot_dir = snapshot_download(repo_id=vae_model_path)
# copy snapshot to model dir
copy_tree(snapshot_dir, str(model_tar.joinpath(vae_model_path)))

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

['model-13382/stabilityai/sd-vae-ft-mse/config.json',
 'model-13382/stabilityai/sd-vae-ft-mse/README.md',
 'model-13382/stabilityai/sd-vae-ft-mse/.gitattributes',
 'model-13382/stabilityai/sd-vae-ft-mse/diffusion_pytorch_model.safetensors',
 'model-13382/stabilityai/sd-vae-ft-mse/diffusion_pytorch_model.bin']

In [12]:
# download snapshot
snapshot_dir = snapshot_download(repo_id=image_encoder_path, allow_patterns=['config.json', 'model.safetensors'])
# copy snapshot to model dir
copy_tree(snapshot_dir, str(model_tar.joinpath(image_encoder_path)))

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

['model-13382/laion/CLIP-ViT-H-14-laion2B-s32B-b79K/.gitattributes',
 'model-13382/laion/CLIP-ViT-H-14-laion2B-s32B-b79K/config.json',
 'model-13382/laion/CLIP-ViT-H-14-laion2B-s32B-b79K/README.md',
 'model-13382/laion/CLIP-ViT-H-14-laion2B-s32B-b79K/open_clip_config.json',
 'model-13382/laion/CLIP-ViT-H-14-laion2B-s32B-b79K/merges.txt',
 'model-13382/laion/CLIP-ViT-H-14-laion2B-s32B-b79K/special_tokens_map.json',
 'model-13382/laion/CLIP-ViT-H-14-laion2B-s32B-b79K/preprocessor_config.json',
 'model-13382/laion/CLIP-ViT-H-14-laion2B-s32B-b79K/tokenizer_config.json',
 'model-13382/laion/CLIP-ViT-H-14-laion2B-s32B-b79K/tokenizer.json',
 'model-13382/laion/CLIP-ViT-H-14-laion2B-s32B-b79K/vocab.json',
 'model-13382/laion/CLIP-ViT-H-14-laion2B-s32B-b79K/model.safetensors']

In [13]:
# download snapshot
snapshot_dir = hf_hub_download(repo_id="h94/IP-Adapter-FaceID", filename=ip_ckpt)
# copy snapshot to model dir
copy_file(snapshot_dir, str(model_tar))

('model-13382/ip-adapter-faceid-plus_sd15.bin', 1)

In [14]:
copy_tree('Example-ML-Solutions-in-AWS/FaceEditorInAWS/code/', str(model_tar.joinpath('code')))

['model-13382/code/pipelines/diffusion_pipelines.py',
 'model-13382/code/pipelines/__init__.py',
 'model-13382/code/.ipynb_checkpoints/inference-checkpoint.py',
 'model-13382/code/.ipynb_checkpoints/requirements-checkpoint.txt',
 'model-13382/code/inference.py',
 'model-13382/code/requirements.txt',
 'model-13382/code/__init__.py',
 'model-13382/code/loaders/ip_adapter_face_plus.py',
 'model-13382/code/loaders/__init__.py',
 'model-13382/code/helper/nn_modules.py',
 'model-13382/code/helper/__init__.py']

In [25]:
!cd {str(model_tar)} && tar zcvf ../model.tar.gz *

code/
code/pipelines/
code/pipelines/diffusion_pipelines.py
code/pipelines/__init__.py
code/.ipynb_checkpoints/
code/.ipynb_checkpoints/inference-checkpoint.py
code/.ipynb_checkpoints/requirements-checkpoint.txt
code/inference.py
code/requirements.txt
code/__init__.py
code/loaders/
code/loaders/ip_adapter_face_plus.py
code/loaders/__init__.py
code/helper/
code/helper/nn_modules.py
code/helper/__init__.py
ip-adapter-faceid-plus_sd15.bin
laion/
laion/CLIP-ViT-H-14-laion2B-s32B-b79K/
laion/CLIP-ViT-H-14-laion2B-s32B-b79K/open_clip_config.json
laion/CLIP-ViT-H-14-laion2B-s32B-b79K/preprocessor_config.json
laion/CLIP-ViT-H-14-laion2B-s32B-b79K/special_tokens_map.json
laion/CLIP-ViT-H-14-laion2B-s32B-b79K/README.md
laion/CLIP-ViT-H-14-laion2B-s32B-b79K/tokenizer.json
laion/CLIP-ViT-H-14-laion2B-s32B-b79K/.gitattributes
laion/CLIP-ViT-H-14-laion2B-s32B-b79K/config.json
laion/CLIP-ViT-H-14-laion2B-s32B-b79K/merges.txt
laion/CLIP-ViT-H-14-laion2B-s32B-b79K/vocab.json
laion/CLIP-ViT-H-14-laion2B

In [26]:
# # helper to create the model.tar.gz
# def compress(tar_dir=None,output_file="model.tar.gz"):
#     parent_dir=os.getcwd()
#     os.chdir(tar_dir)
#     with tarfile.open(os.path.join(parent_dir, output_file), "w:gz") as tar:
#         for item in os.listdir('.'):
#             print(item)
#             tar.add(item, arcname=item)
#     os.chdir(parent_dir)

# compress(str(model_tar))

In [27]:
# upload model.tar.gz to s3
s3_model_uri=S3Uploader.upload(local_path="model.tar.gz", desired_s3_uri=f"s3://{sess.default_bucket()}/face_editor")

print(f"model uploaded to: {s3_model_uri}")

model uploaded to: s3://sagemaker-eu-north-1-564976835481/face_editor/model.tar.gz


In [28]:
# create Hugging Face Model Class
huggingface_model = HuggingFaceModel(
   model_data=s3_model_uri,      # path to your model and script
   role=role,                    # iam role with permissions to create an Endpoint
   transformers_version='4.28',  # transformers version used
   pytorch_version='2.0',        # pytorch version used
   py_version='py310',           # python version used
)

In [29]:
predictor = huggingface_model.deploy(
    initial_instance_count=1,
    instance_type="ml.g4dn.xlarge"
    )

-----------!

# Test Endpoint

In [7]:
from PIL import Image
from io import BytesIO
from IPython.display import display
import base64
import cv2
import boto3
import numpy as np
import json
import matplotlib.pyplot as plt

# helper decoder
def decode_base64_image(image_string):
    base64_image = base64.b64decode(image_string)
    buffer = BytesIO(base64_image)
    return Image.open(buffer)

# display PIL images as grid
def display_images(images=None,columns=3, width=100, height=100):
    plt.figure(figsize=(width, height))
    for i, image in enumerate(images):
        plt.subplot(int(len(images) / columns + 1), columns, i + 1)
        plt.axis('off')
        plt.imshow(image)

In [8]:
buffered = BytesIO()
Image.open("samples/Screenshot from 2024-02-09 13-53-24.png").save(buffered, format="PNG")
image_string = base64.b64encode(buffered.getvalue()).decode()

In [19]:
data={
    "prompt":"a beautiful girl , high quality, 8k",
    "image": image_string,
    "num_inference_steps":30,
    "num_images_per_prompt":4
  }

In [20]:
# if predictor is not here then you may call your endpoint through name
is_predictor_loaded = False

In [21]:
# run prediction
if is_predictor_loaded:
    response = predictor.predict(data = data)
else:
    client = boto3.client("sagemaker-runtime")
    payload = json.dumps(data)
    endpoint_result = client.invoke_endpoint(
        EndpointName="huggingface-pytorch-inference-2024-02-09-12-13-11-335",
        ContentType="application/json",
        Body=payload, )
    response = eval(endpoint_result["Body"].read())

In [23]:
# decode images
decoded_images = [decode_base64_image(image) for image in response["generated_images"]]

# visualize generation
display_images(decoded_images)

In [24]:
predictor.delete_model()
predictor.delete_endpoint()